# 데이터 구조 이해 & SQL 스키마 설계
- JSON 데이터 로드 및 컬럼 확인 (pandas)
- 결측치·중복 데이터 확인/정제
- SQL DB 스키마 설계 (Supplements, Ingredients, Supplement_Ingredients, Symptoms_Ingredients)• 증상–성분 매핑 테이블 초안 작성

- json 파일 데이터 개수: 1241개, 23열 ( )

In [ ]:
import pandas as pd
import numpy as np

In [291]:

# 1) 로드
fp = '/Users/gim-yujin/Desktop/pjt_personal_agent/영양소 데이터/iherb_data_uk_data_2022_12.json'
df = pd.read_json(fp, orient='records')   # 파일이 리스트 of dicts 여야 정상
# 2) 전체 컬럼 확인
print(df.shape)
print(df.columns.tolist())
# 3) 샘플 확인
display(df.head())
# 4) 기본 타입 정리
df['scraped_at'] = pd.to_datetime(df['scraped_at'], dayfirst=True, errors='coerce')

(1241, 23)
['P url', 'Title', 'Product code', 'Price', 'Currency', 'UPC Code', 'Pid', 'Shipping Weight', 'Dimensions', 'Category 1', 'Category 2', 'Category 3', 'Availability', 'Brand', 'images', 'Product specifications', 'Description', 'ingredients', 'Supplement Facts', 'Avg rating', 'Reviews count', 'uniq_id', 'scraped_at']


,P url,Title,Product code,Price,Currency,UPC Code,Pid,Shipping Weight,Dimensions,Category 1,...,Brand,images,Product specifications,Description,ingredients,Supplement Facts,Avg rating,Reviews count,uniq_id,scraped_at
0,https://uk.iherb.com/pr/medinatura-wellmind-ca...,"MediNatura, WellMind Calming Day/Night, 100 Ta...",787647704033,11.38,GBP,HEE-70403,8175,0.05 kg,5.3 x 5.3 x 9.7 cm,MediNatura,...,MediNatura,https://cloudinary.images-iherb.com/image/uplo...,"Date First Available: May 30 2007, Shipping W...",From the Makers of T-Relief™Day - Natural Calm...,Drug FactsActive Ingredient**Purpose*Chamomill...,,4.5,641,e747b7b0-2990-5d30-ade6-d32315f04574,2022-12-15 12:39:32
1,https://uk.iherb.com/pr/wisdom-natural-sweetle...,"Wisdom Natural, SweetLeaf, Sweet Drops Stevia ...",716123125659,15.92,GBP,WDN-12565,5106,0.38 kg,4.8 x 4.8 x 13.2 cm,Wisdom Natural,...,Wisdom Natural,https://cloudinary.images-iherb.com/image/uplo...,Best By: June 1 2024\n \n\n\n\n...,No SugarsNo Artificial IngredientsNon GMO Proj...,"Purified water, organic stevia leaf extract, n...",Supplement FactsServing Size: 5 drops (0.208 m...,4.5,827,2cac8d45-6138-5c72-a3c5-2ed96cb61e90,2022-12-15 12:39:33
2,https://uk.iherb.com/pr/enough-gold-snail-mois...,"Enough, Gold Snail, Moisture Foundation, SPF 3...",8809474498809,6.28,GBP,ENO-49880,112679,0.17 kg,15.1 x 4.1 x 3.9 cm,Enough,...,Enough,https://cloudinary.images-iherb.com/image/uplo...,Best By: June 23 2025\n \n\n\n\...,This foundation has excellent covering propert...,"Water, cyclopentasiloxane, titanium dioxide, c...",,4.5,72,a6fac746-ddac-5c1e-86ab-5df2058bb2ea,2022-12-15 12:39:35
3,https://uk.iherb.com/pr/natrol-b-complex-fast-...,"Natrol, B-Complex, Fast Dissolve, Coconut, 90 ...",47469063313,9.79,GBP,NTL-06331,45924,0.16 kg,11.4 x 5.8 x 5.8 cm,Natrol,...,Natrol,https://cloudinary.images-iherb.com/image/uplo...,Best By: June 30 2024\n \n\n\n\...,Energy Support Promotes EnergySupports a Healt...,"Xylitol, maltodextrin, dextrose, cellulose gum...",Supplement FactsServing Size: 1 TabletServings...,4.8,1326,77b8fdb5-d02a-58b1-9cb4-6485791249b3,2022-12-15 12:39:37
4,https://uk.iherb.com/pr/artnaturals-luxe-avoca...,"artnaturals, Luxe, Avocado Oil Shampoo, Dry Ha...",810029800242,13.71,GBP,RNA-80024,110416,0.64 kg,18.9 x 6.7 x 6.6 cm,artnaturals,...,artnaturals,https://cloudinary.images-iherb.com/image/uplo...,Best By: January 1 2025\n \n\n\...,Moisture Strength & ShineNot Tested on Animals...,"Water, potassium sorbate, sodium benzoate, tet...",,4.6,38,8206e129-7b4b-5715-9149-a28ea886470a,2022-12-15 12:39:37


In [ ]:
print("##컬럼명 목록")
print(df.columns)
print("-" * 50)
# 컬럼별 결측치 개수 확인 
print("##컬럼별 결측치 개수 확인(공백 문자열이 결측치로 합산이 안되어 모두 0으로 표기됨)")
print(df.isnull().sum())

In [294]:
df[['Supplement Facts']]

,Supplement Facts
0,
1,Supplement FactsServing Size: 5 drops (0.208 m...
2,
3,Supplement FactsServing Size: 1 TabletServings...
4,
...,...
1236,
1237,Supplement FactsServing Size: 2 Vegetable Caps...
1238,Supplement FactsServing Size: 2 GummiesServing...
1239,Supplement FactsServing Size: One Capsule Amou...


### 결측치 확인
- 결측값이 존재하는 컬럼 및 개수
- Category 2            4
- Category 3          869
- ingredients          61
- Supplement Facts    530 (추후에 성분을 참고하여 채워 넣을 예정, 보충정보(영양성분))

In [ ]:
# (선택) 모든 컬럼에 대해 한 번에 적용할 수도 있습니다.
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
print("\n## 전체 데이터의 실제 결측치 수")
print(df.isnull().sum())

In [ ]:
#  각 컬럼의 결측치 개수를 계산
missing_values = df.isnull().sum()
#  결측치 개수가 0보다 큰 컬럼들만 필터링하여 출력합니다.
columns_with_missing_values = missing_values[missing_values > 0]

print("## 결측값이 존재하는 컬럼 및 개수")
if columns_with_missing_values.empty:
    print("모든 컬럼의 데이터가 채워져 있습니다.")
else:
    print(columns_with_missing_values)

In [ ]:
#결측값이 존재하는 컬럼 선택 출력 확인 

df_missing = df[columns_with_missing_values.index]
print(df_missing)

In [ ]:
df[columns_with_missing_values.index]

### 중복 제거 

- unique_id 기준(0)
- Pid, title 기준(0)
- 중복되는 아이템은 없음


In [ ]:
# 중복 제거 (uniqe_id 기준으로는 중복 없음.)
df = df.drop_duplicates(subset=['uniq_id'])  
# uniq_id가 있으면 안전
df.drop_duplicates(subset=['Pid','Title'])
print(df)

In [ ]:
df.drop_duplicates(subset=['Pid','Title']) #여기도 중복은 없는 것으로 확인 

In [ ]:
df['Price'] = df['Price'].astype(str)
# price >> 문자열로 공백/ 쉼표 제고후 float
df['Price'] = df['Price'].str.replace(',', '').str.strip()
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')


### 불필요한 레코드 필터링 
- 카테고리 1, 2 기준으로 키워드 필터링 한 결과, 해당 데이터셋에서 영양제와 관련된 상품 개수는 1221->325 개로 감소함.


In [ ]:
## 화장품/ 샴푸/ 식품/ 베이비 용품 등 필터링 
non_supp_cats = ['Shampoo','Foundation','Face Wash','Utensils','Diapers']  # 예시
df = df[~df['Category 2'].isin(non_supp_cats)]
df.count

In [ ]:
# 'Category 1' 컬럼의 모든 고유값 출력
print("Category 1의 고유값:", df['Category 1'].unique())

# 'Category 2' 컬럼의 모든 고유값 출력
print("Category 2의 고유값:", df['Category 2'].unique())

# 'Category 3' 컬럼의 모든 고유값 출력
print("Category 3의 고유값:", df['Category 3'].unique())

In [ ]:
## 화이트리스트 키워드(supp_keywords)

supp_keywords = [
    # 비타민·미네랄
    'vitamin', 'multivitamin', 'multimineral', 'b1', 'b2', 'b3', 'b6',
    'b12', 'c', 'd', 'e', 'k', 'folic acid', 'niacin', 'biotin',
    'calcium', 'magnesium', 'zinc', 'iron', 'selenium', 'potassium',
    'iodine', 'trace minerals',

    # 오메가 & 필수지방산
    'omega', 'fish oil', 'krill oil', 'cod liver oil',
    'efa', 'dha', 'epa',

    # 허브·식물 추출물
    'herb', 'herbal', 'ashwagandha', 'ginseng', 'echinacea', 'turmeric',
    'curcumin', 'milk thistle', 'rhodiola', 'elderberry', 'boswellia',
    'sambucus', 'hawthorn', 'garlic', 'ginger', 'licorice', 'oregano',
    'passion flower', 'valerian', 'chamomile', 'nettle', 'schisandra',
    'astragalus',

    # 아미노산·단백질
    'amino', 'amino acid', 'l-',   # L-Arginine, L-Tyrosine 등 앞에 L-이 붙음
    'protein', 'collagen', 'peptide',

    # 프로바이오틱/소화
    'probiotic', 'prebiotic', 'lactobacillus', 'bifidus',
    'digestive enzymes', 'enzyme',

    # 항산화·기타 보조성분
    'coq10', 'ubiquinol', 'alpha lipoic acid', 'resveratrol',
    'pycnogenol', 'glutathione', 'chlorophyll', 'spirulina',
    'chlorella', 'maca', 'bee pollen', 'royal jelly',

    # 특수 목적 포뮬러
    'immune', 'immune support', 'energy formula', 'sleep formula',
    'cognitive', 'memory', 'joint', 'bone', 'liver', 'thyroid',
    'blood support', 'heart support', 'detox', 'women\'s health',
    'men\'s health', 'prenatal', 'post-natal',
    'sports supplement', 'workout', 'weight management', 'fat burner',

    # 형태·일반명
    'supplement', 'dietary', 'nutrition', 'nutrient',
    'superfood', 'greens', 'superfood blend'
]


In [ ]:
import re

def is_supplement_row(row):
    cats = " ".join([
        str(row.get('Category 2', '')).lower(),
        str(row.get('Category 3', '')).lower()
    ])
    return any(re.search(rf"\b{k}\b", cats) for k in supp_keywords)

df_supp = df[df.apply(is_supplement_row, axis=1)].copy()

print(f"필터 전 {len(df)} → 필터 후 {len(df_supp)}")

### 제대로 필터링 되었는지 확인 작업
- 멀티비타민과, 비타민 구분하여 놓았는지 

In [ ]:
import re

# 1️⃣ 화이트리스트 기반 영양제 필터
def is_supplement_row(row):
    cats = " ".join([
        str(row.get('Category 2', '')).lower(),
        str(row.get('Category 3', '')).lower()
    ])
    return any(re.search(rf"\b{k}\b", cats) for k in supp_keywords)

df_supp = df[df.apply(is_supplement_row, axis=1)].copy()

# 2️⃣ 블랙리스트 제거 (현재는 K-Beauty 하나지만 확장 가능)
black_keywords = ['k-beauty']
def not_blacklisted(row):
    cats = " ".join([
        str(row.get('Category 1', '')).lower(),
        str(row.get('Category 2', '')).lower(),
        str(row.get('Category 3', '')).lower()
    ])
    return not any(bk in cats for bk in black_keywords)

df_supp = df_supp[df_supp.apply(not_blacklisted, axis=1)].copy()

print(f"최종 필터 후 행 수 : {len(df_supp)}")

In [ ]:
# 3️⃣ 무작위 100건 추출 (중복 없이)
sample_100 = df_supp.sample(n=100, random_state=42)  # random_state는 재현성

# 4️⃣ 검토에 유용한 컬럼만 보기
cols_to_check = ['Title', 'Category 1', 'Category 2', 'Category 3', 'Description']
print(sample_100[cols_to_check].to_string(index=False))



### “파싱(parsing)”: 문자열(예: Supplement Facts 텍스트) 안에서 우리가 원하는 정보(성분명, 용량, 단위 등)를 규칙적으로 뽑아내는 작업

- 1.	parse_supplement_facts()
→ 텍스트를 줄 단위로 읽고 정규식으로 [성분, 수치, 단위] 추출.
- 2.	parse_and_flag()
→ 전체 DataFrame에 적용, parsed_ingredients와 parse_error 컬럼 추가.
- 3.	수동 검토
→ parse_error=True인 레코드만 CSV로 내보내어 직접 확인·수정.
--- 
- df_supp_checked

- parsed_ingredients: 파싱 성공 시 [{'name':…, 'amount':…, 'unit':…}, …] 리스트

- parse_error: True(실패) / False(성공)

- supplement_parse_errors.csv

사람이 직접 살펴보고 정규식 보완이나 데이터 수동 입력이 필요한 상품 목록.


In [ ]:
keyword_pattern = r'(?i)' + '|'.join([re.escape(k) for k in supp_keywords])

In [ ]:
def parse_supplement_facts(text):
    """
    Supplement Facts 문자열에서
    [성분명, 수치, 단위] 추출
    """
    results = []
    if not isinstance(text, str) or not text.strip():
        return results  # 빈 값이면 바로 실패
    
    # 예) "Vitamin C 500 mg", "Magnesium (as oxide) 250 mg"
    pattern = r'([A-Za-z0-9\-\(\) /]+?)\s+([\d.,]+)\s*(mg|mcg|µg|g|iu|IU)'
    
    for line in text.splitlines():
        m = re.search(pattern, line)
        if m:
            name = m.group(1).strip()
            amount = float(m.group(2).replace(',', ''))
            unit = m.group(3).lower()
            results.append({
                'name': name,
                'amount': amount,
                'unit': unit,
                'raw_line': line.strip()
            })
    return results

In [258]:
def parse_and_flag_supp(df_supp):
    parsed_results = []
    parse_error_flags = []

    for text in df_supp['Supplement Facts']:
        parsed = parse_supplement_facts(text)
        parsed_results.append(parsed)
        # 파싱 결과가 없으면 True → 실패
        parse_error_flags.append(len(parsed) == 0)

    df_supp_checked = df_supp.copy()
    df_supp_checked['parsed_ingredients'] = parsed_results
    df_supp_checked['parse_error'] = parse_error_flags
    return df_supp_checked

# ✅ 실행
df_supp_checked = parse_and_flag_supp(df_supp)


In [259]:
len(df_supp_checked)

325

In [260]:

# 파싱 실패 데이터만 추출
df_supp_errors = df_supp_checked[df_supp_checked['parse_error']]

# 주요 컬럼만 저장
cols_to_review = ['Title', 'Category 2', 'Category 3', 'Supplement Facts']
df_supp_errors[cols_to_review].to_csv('supplement_parse_errors.csv', index=False)

print(f"파싱 실패 레코드 수: {len(df_supp_errors)}")
print("수동 검토 파일: supplement_parse_errors.csv 저장 완료!")


파싱 실패 레코드 수: 0
수동 검토 파일: supplement_parse_errors.csv 저장 완료!


In [261]:
df_supp_errors = df_supp_checked[df_supp_checked['parse_error']]
print("파싱 실패 레코드 수:", len(df_supp_errors))
df_supp_errors[['Title','Category 2','Category 3','Supplement Facts']].head(10)

파싱 실패 레코드 수: 0


,Title,Category 2,Category 3,Supplement Facts


### 파싱 실패 레코드 처리 
- HTML 태그 제거

- Proprietary / Matrix / Blend 감지

- 다중 %DV 항목 탐지

- Serving Size, Amount Per Serving 기반 구조성 여부 판단

결과: parsed, 또는 실패한 경우 실패 사유 코드 리턴 -->
1.	다양한 줄바꿈 (\n, \r\n) 혹은 공백을 제거하여 일관성 있게 처리
2.	Serving Size, Amount Per Serving, % Daily Value 등 핵심 키워드를 기준으로 텍스트를 구조화
3.	영양소 정보 블록을 정확히 추출
4.	Markdown 또는 HTML 태그, 기호 (†, ‡) 제거
5.	공란 또는 비정상 케이스에 대해 안전하게 예외 처리
--

In [ ]:

# 1. 데이터 불러오기
df = pd.read_csv("supplement_parse_errors.csv")

# 2. 실패 이유 판별 함수
def classify_fail_reason(row):
    fact = str(row.get("Supplement Facts", "")).strip().lower()

    if not fact or fact == "nan":
        return "missing_fact"

    if re.search(r"(<br>|\\n|\\r|^\s+|\n\s*\n)", fact):
        return "html_formatting"

    if re.search(r"proprietary|herbal blend|extract|complex", fact):
        return "proprietary_blend"

    if re.search(r"%\s*(dv|daily value)[^%]+%.*(child|children|adults|1-3|4+)", fact):
        return "multi_dv"

    if "serving size" in fact and not re.search(r"amount per serving|% daily value|% dv", fact):
        return "unstructured"

    return "other"

# 3. 적용
df["fail_reason"] = df.apply(classify_fail_reason, axis=1)

# 4. 결과 확인 (상위 10개)
print(df[["Title", "fail_reason"]].head(30))

# 5. 저장 (선택 사항)
df.to_csv("classified_parse_errors.csv", index=False)


In [ ]:
# 보완된 파싱 함수(텍스트 기반이지만 규칙적으로 나열된 성분 정보 뽑기)

def parse_supplement_facts(text: str) -> dict:
    if not text or not isinstance(text, str):
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}

    cleaned = text.strip()

    # 핵심 키워드로 시작점 잡기
    start_keywords = ['Supplement Facts', 'Serving Size']
    start_index = -1
    for keyword in start_keywords:
        if keyword in cleaned:
            start_index = cleaned.find(keyword)
            break

    if start_index == -1:
        return {"status": "fail", "reason": "핵심 키워드 없음", "data": None}

    # 줄바꿈 및 특수 문자 제거
    block = cleaned[start_index:]
    block = re.sub(r'[\n\r\t]', ' ', block)
    block = re.sub(r'†|‡|[*]+|[%]+', '', block)
    block = re.sub(r'\s{2,}', ' ', block).strip()

    # 다양한 단위 포함 (비표준 단위 대응 포함)
    unit_pattern = r'mg|mcg|µg|g|iu|IU|ALU|HUT|FCCFIP|DP°|XU|GalU|AGU|CFU|DPPU|SU|CU|Endo-PGU|HCU|FIP|mg\*|IU\*|g\*'
    
    # 성분 추출: "Vitamin C 500 mg", "Lactase 9500 ALU", "CoQ10 200 mg"
    pattern = rf'([A-Za-z0-9®,\-\(\)\'\"\+/\. ]{{2,}}?)\s+([\d,\.]+)\s*({unit_pattern})?'
    matches = re.findall(pattern, block)

    if not matches:
        return {"status": "fail", "reason": "성분 추출 실패", "data": None}

    nutrients = []
    for name, value, unit in matches:
        name_clean = name.strip().replace(":", "").replace("†", "")
        amount = value.replace(",", "")
        nutrients.append({
            "name": name_clean,
            "amount": amount,
            "unit": unit or ""
        })

    return {"status": "success", "count": len(nutrients), "data": nutrients}


In [ ]:
example_text = """
Supplement Facts
Serving Size: 1 Tablet
Amount Per Serving % Daily Value
Vitamin C (as Ascorbic Acid) 500 mg 833%
Zinc (as Zinc Gluconate) 15 mg 136%
"""

result = parse_supplement_facts(example_text)
print(result)

In [ ]:
def reparse_failed(df_checked):
    reparsed = []
    reasons = []

    for idx, row in df_checked.iterrows():
        if not row['parse_error']:
            reparsed.append(row['parsed_ingredients'])
            reasons.append("정상 파싱")
        else:
            result = parse_supplement_facts_v3(row['Supplement Facts'])
            if result["status"] == "success":
                reparsed.append(result['data'])
                reasons.append("보완 파싱 성공")
            else:
                reparsed.append(None)
                reasons.append(result['reason'])

    df_checked['parsed_final'] = reparsed
    df_checked['fail_reason_final'] = reasons
    df_checked['final_parse_error'] = df_checked['parsed_final'].isnull()
    return df_checked

In [ ]:
# 실패 사유별 개수
fail_summary = df_final_errors['fail_reason_final'].value_counts()
print("📌 보완된 파싱 이후 실패 사유 통계:\n", fail_summary)

In [ ]:
## 최종 보완용 파싱 함수 만들기 
def parse_supplement_facts_flexible(text: str) -> dict:
    if not text or not isinstance(text, str):
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}

    # 텍스트 정리
    block = text.strip()
    block = re.sub(r'[\n\r\t]', ' ', block)
    block = re.sub(r'†|‡|[*]+|[%]+', '', block)
    block = re.sub(r'\s{2,}', ' ', block).strip()

    # 패턴: 성분명 (value) (단위) → 단위 생략도 허용
    pattern = r'([A-Za-z0-9 \-\(\)\[\]/]+?)\s+([\d,\.]+)\s*(mg|mcg|g|IU|iu|µg|mcg|ml|capsules|tablets|softgels|veggie capsules)?'

    matches = re.findall(pattern, block)

    if not matches:
        return {"status": "fail", "reason": "성분 추출 실패", "data": None}

    parsed = []
    for name, amount, unit in matches:
        parsed.append({
            "name": name.strip(),
            "amount": amount.strip(),
            "unit": (unit or "").lower()
        })

    return {"status": "success", "count": len(parsed), "data": parsed}


In [ ]:
# 위의 함수를 활용해서 재파싱 실행 

def apply_final_flexible_parsing(df):
    results = []
    errors = []
    reasons = []

    for text in df['Supplement Facts']:
        res = parse_supplement_facts_flexible(text)
        results.append(res)
        errors.append(res['status'] == 'fail')
        reasons.append(res['reason'] if res['status'] == 'fail' else '')

    df = df.copy()
    df['parse_result_final2'] = results
    df['parse_error_final2'] = errors
    df['fail_reason_final2'] = reasons
    return df

In [ ]:

# 적용
df_final2 = apply_final_flexible_parsing(df_final_errors)

# 여전히 실패한 데이터
df_final2_errors = df_final2[df_final2['parse_error_final2']]

# 결과 요약
fail_summary2 = df_final2_errors['fail_reason_final2'].value_counts()
print("📌 최종 보완 파싱 이후 실패 사유:\n", fail_summary2)

# 수동 검토용 저장
df_final2_errors[['Title', 'Category 2', 'Category 3', 'Supplement Facts']].to_csv('supplement_parse_errors_final2.csv', index=False)

In [ ]:
### 진짜 최종 파싱 보완 함수

import re

def parse_supplement_facts_v3(text: str) -> dict:
    if not text or not isinstance(text, str):
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}

    cleaned = text.strip()

    # 시작 지점 추정
    start_keywords = ['Supplement Facts', 'Amount Per Serving']
    start_index = -1
    for keyword in start_keywords:
        if keyword in cleaned:
            start_index = cleaned.find(keyword)
            break

    if start_index == -1:
        return {"status": "fail", "reason": "핵심 키워드 없음", "data": None}

    block = cleaned[start_index:]
    block = re.sub(r'[\n\r\t]', ' ', block)
    block = re.sub(r'†|‡|[*]+|[%]+|††', '', block)
    block = re.sub(r'\s{2,}', ' ', block).strip()

    # 다양한 단위 포함 (비표준 단위 대응 포함)
    unit_pattern = r'mg|mcg|µg|g|iu|IU|ALU|HUT|FCCFIP|DP°|XU|GalU|AGU|CFU|DPPU|SU|CU|Endo-PGU|HCU|FIP|mg\*|IU\*|g\*'
    
    # 성분 추출: "Vitamin C 500 mg", "Lactase 9500 ALU", "CoQ10 200 mg"
    pattern = rf'([A-Za-z0-9®,\-\(\)\'\"\+/\. ]{{2,}}?)\s+([\d,\.]+)\s*({unit_pattern})?'
    matches = re.findall(pattern, block)

    if not matches:
        return {"status": "fail", "reason": "성분 추출 실패", "data": None}

    nutrients = []
    for name, value, unit in matches:
        name_clean = name.strip().replace(":", "").replace("†", "")
        amount = value.replace(",", "")
        nutrients.append({
            "name": name_clean,
            "amount": amount,
            "unit": unit or ""
        })

    return {"status": "success", "count": len(nutrients), "data": nutrients}


In [ ]:
## 실패 데이터에 최종 파싱 적용
def reparse_failed(df_checked):
    reparsed = []
    reasons = []

    for idx, row in df_checked.iterrows():
        if not row['parse_error']:
            reparsed.append(row['parsed_ingredients'])
            reasons.append("정상 파싱")
        else:
            result = parse_supplement_facts_v3(row['Supplement Facts'])
            if result["status"] == "success":
                reparsed.append(result['data'])
                reasons.append("보완 파싱 성공")
            else:
                reparsed.append(None)
                reasons.append(result['reason'])

    df_checked['parsed_final'] = reparsed
    df_checked['fail_reason_final'] = reasons
    df_checked['final_parse_error'] = df_checked['parsed_final'].isnull()
    return df_checked

In [ ]:
# 보완된 파서 (V3)를 적용한 최종 파싱 함수
def final_parse_supplement(text):
    result = parse_supplement_facts_v3(text)
    return result

def final_flag_parse_error(parse_result):
    if isinstance(parse_result, dict) and parse_result.get("status") == "fail":
        return True
    return False

def final_get_fail_reason(parse_result):
    if isinstance(parse_result, dict) and parse_result.get("status") == "fail":
        return parse_result.get("reason")
    return None

# ✅ df_supp는 원본 데이터프레임 (또는 전처리된 데이터프레임)
df_final = df_supp.copy()

# 최종 파싱 결과 적용
df_final["final_parse_result"] = df_final["Supplement Facts"].apply(final_parse_supplement)
df_final["final_parse_error"] = df_final["final_parse_result"].apply(final_flag_parse_error)
df_final["fail_reason_final"] = df_final["final_parse_result"].apply(final_get_fail_reason)

In [ ]:
# 최종 파싱 실패 레코드 수 출력
num_final_failures = df_final['final_parse_error'].sum()
print(f"❌ 최종 파싱 실패한 레코드 수: {num_final_failures}개")

# 실패 사유별 분포도 확인
fail_summary = df_final['fail_reason_final'].value_counts()
print("\n📊 파싱 실패 사유 분포:")
print(fail_summary)

In [ ]:
# ❗최종 파싱 실패한 레코드만 저장
df_final_errors = df_final[df_final['final_parse_error'] == True]

# 주요 컬럼만 추출해서 저장
df_final_errors[['Title', 'Category 2', 'Category 3', 'Supplement Facts']].to_csv(
    'supplement_parse_errors_final.csv', index=False
)

print(f"📁 supplement_parse_errors_final.csv 저장 완료! 실패한 레코드 수: {len(df_final_errors)}개")

In [ ]:
### 최최최종 파싱 
import re

def final_parse_supplement_facts(text):
    if not isinstance(text, str) or not text.strip():
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}
    
    # 전처리
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'[*†‡]+', '', text)
    text = re.sub(r'\s{2,}', ' ', text).strip()

    # 주석 구간 제거 (Daily Value not established 등)
    text = re.sub(r'Daily Value.*?established[.]*', '', text, flags=re.IGNORECASE)

    # 패턴: 성분명 + 수치 + 단위
    pattern = r'([A-Za-z0-9®,\-’\'\"\(\)\[\]\/\+\:\s]+?)\s+([\d\.,]+)\s*(mcg|µg|mg|g|iu|IU|%)'

    matches = re.findall(pattern, text)

    if not matches:
        return {"status": "fail", "reason": "성분 추출 실패", "data": None}

    results = []
    for name, value, unit in matches:
        try:
            value = float(value.replace(',', '').strip())
        except:
            continue
        results.append({
            'name': name.strip(),
            'amount': value,
            'unit': unit.lower()
        })

    return {
        "status": "success",
        "count": len(results),
        "data": results
    }

In [ ]:
# 예시 텍스트
example = """
Supplement Facts
Serving Size: 1 Tablet
Amount Per Serving % Daily Value
Vitamin C (as Ascorbic Acid) 500 mg 833%
Zinc (as Zinc Gluconate) 15 mg 136%
Biotin 333 mcg 1,110%
"""

result = final_parse_supplement_facts(example)
print(result)

In [ ]:
df['parse_result'] = df['Supplement Facts'].apply(final_parse_supplement_facts)
df['parse_status'] = df['parse_result'].apply(lambda x: x['status'])
df['fail_reason'] = df['parse_result'].apply(lambda x: x['reason'] if x['status'] == 'fail' else None)

# 실패 데이터만 저장
df_errors = df[df['parse_status'] == 'fail']
df_errors[['Title', 'Category 2', 'Category 3', 'Supplement Facts', 'fail_reason']].to_csv('supplement_parse_errors_final3.csv', index=False)

In [ ]:
# 'fail_reason_final'이 있는 경우
fail_counts = df_final['fail_reason_final'].value_counts()
print("📊 실패 사유별 개수:\n", fail_counts)

In [ ]:
# ## 흐름 정리 
# # 1. 먼저 함수 정의
# def parse_supplement_facts_v3(text): ...
# def apply_final_parsing(df): ...

# # 2. 데이터프레임에 적용
# df_final = apply_final_parsing(df_failed)  # df_failed는 실패한 레코드 모음

# # 3. 실패한 개수 및 이유 확인
# df_final['fail_reason_final'].value_counts()

In [ ]:
## 절망적이어도 가자.. 4차 보완 파싱 함수 

import re

def parse_supplement_facts_v3(text: str) -> dict:
    if not text or not isinstance(text, str) or text.strip() == "":
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}

    cleaned = text.strip()
    block = re.sub(r'[\n\r\t]', ' ', cleaned)
    block = re.sub(r'†|‡|[*]+|[%]+', '', block)
    block = re.sub(r'\s{2,}', ' ', block).strip()

    # 보완된 정규식: 단위 다양화 + 괄호 안 성분 허용
    pattern = r'([A-Za-z0-9 \-–®™\(\)\[\],\'+°]+?)\s+([\d\.,]+)\s*(mcg|mg|g|iu|IU|%)?'

    matches = re.findall(pattern, block)
    if not matches:
        return {"status": "fail", "reason": "성분 추출 실패", "data": None}

    nutrients = []
    for name, amount, unit in matches:
        try:
            nutrients.append({
                "name": name.strip(),
                "amount": float(amount.replace(",", "")),
                "unit": unit or ""
            })
        except:
            continue

    return {
        "status": "success" if nutrients else "fail",
        "reason": None if nutrients else "성분 추출 실패",
        "count": len(nutrients),
        "data": nutrients if nutrients else None
    }

In [ ]:
# 통합 적용 함수 
def apply_final_parsing(df):
    parsed_results = []
    parse_status = []
    fail_reasons = []

    for text in df['Supplement Facts']:
        result = parse_supplement_facts_v3(text)
        parsed_results.append(result['data'])
        parse_status.append(result['status'] == 'fail')
        fail_reasons.append(result['reason'] if result['status'] == 'fail' else None)

    df_result = df.copy()
    df_result['final_parsed'] = parsed_results
    df_result['final_parse_error'] = parse_status
    df_result['fail_reason_final'] = fail_reasons

    return df_result


In [ ]:
# 1️⃣ 초기 파싱 후 실패한 데이터만 추출
df_failed = df_supp_checked[df_supp_checked['parse_error']]

# 2️⃣ 보완된 파싱 함수 적용
df_final = apply_final_parsing(df_failed)

# 3️⃣ 최종 실패한 것 확인
df_final_errors = df_final[df_final['final_parse_error']]
print(f"❌ 최종 실패: {len(df_final_errors)}개")

- 텍스 자체가 난해하거나, 보충제가 아닌 상품이 섞여있어 위의 많은 시도에서 실패한 것임

In [ ]:
import re
import pandas as pd

# ---------- 유틸/정규식 ----------
UNITS_PATTERN = r"(mcg|µg|mg|g|kg|IU|iu|CFU|DPPU|ALU|HUT|FIP|FCCFIP|SU|XU|AGU|DP°?|CU|PGU|HCU|mEq)"
DENY_HEADER = re.compile(
    r'\b(Serving Size|Servings? Per Container|Amount Per Serving|% ?Daily Value|% ?DV|Daily Value|DV\b|Calories\b|'
    r'Total Fat\b|Saturated Fat\b|Trans Fat\b|Cholesterol\b|Sodium\b|Total Carbohydrate\b|Dietary Fiber\b|Total Sugars\b|'
    r'Added Sugars\b|Protein\b|Potassium\b|Calcium\b|Iron\b|Vitamin D\b|Magnesium\b|Phosphorus\b|Manganese\b)\b',
    re.I
)

SKIP_CATS = [
    # 비-보충제로 간주: 파싱 실패 카운트에서 제외
    'tea','herbal tea','peppermint tea','chamomile','black tea',
    'serum','serums','beauty','lotion','face','peel','mask','body','skin',
    'oil','oils',
    'spice','spices','seasoning',
    'bar','bars','protein bar','whey protein bars','milk protein bars',
    'workout enhancer','workout'
]

def normalize_panel(text: str) -> str:
    s = text if isinstance(text, str) else str(text)
    # 유니코드 공백/특수문자 정리
    s = s.replace("\xa0"," ").replace("\u2009"," ").replace("\u202f"," ")
    s = re.sub(r'[\r\t]', ' ', s)
    # †, ‡, *, 불릿 제거
    s = re.sub(r'[†‡*•●]+', '', s)
    # 글자/괄호] 바로 뒤에 숫자가 붙은 경우 공백 삽입: "Niacin250" -> "Niacin 250"
    s = re.sub(r'(?<=[A-Za-z\)\]])(?=\d)', ' ', s)
    # 단위와 다음 숫자 붙은 케이스 공백: "mg1,563%" -> "mg 1,563%"
    s = re.sub(r'(?i)\b(mcg|µg|mg|g|iu|IU|CFU)(?=\d)', r'\1 ', s)
    # 다중 공백 정리
    s = re.sub(r'\s{2,}', ' ', s).strip()
    return s

def parse_line_items(block: str):
    items = []

    # 1) 기본 패턴: [이름] [수치] [단위]
    pat_main = re.compile(
        rf"(?P<name>[A-Za-z][A-Za-z0-9 \-–®™\(\)\[\],\'\+°/\.]+?)\s+"
        rf"(?P<amount>[\d][\d,\.]*)\s*"
        rf"(?P<unit>{UNITS_PATTERN})\b"
    )

    for m in pat_main.finditer(block):
        name = m.group('name').strip()
        if DENY_HEADER.search(name):
            continue
        try:
            amount = float(m.group('amount').replace(',', ''))
            unit = m.group('unit')
            items.append({'name': name, 'amount': amount, 'unit': unit})
        except:
            pass

    # 2) CFU(백만/억 단위) 추가 포착: "(65 Billion CFU)" 등
    pat_cfu = re.compile(r"(?P<amount>[\d][\d,\.]*)\s*(?P<scale>Billion|Million)\s*CFU", re.I)
    for m in pat_cfu.finditer(block):
        try:
            amt = float(m.group('amount').replace(',', ''))
            factor = 1e9 if m.group('scale').lower() == 'billion' else 1e6
            items.append({'name': 'Probiotic CFU', 'amount': amt*factor, 'unit': 'CFU'})
        except:
            pass

    # 3) 중복 제거
    seen = set()
    uniq = []
    for it in items:
        key = (it['name'].lower(), it['unit'].lower(), it['amount'])
        if key in seen:
            continue
        seen.add(key)
        uniq.append(it)
    return uniq

def is_skip_category(cat2, cat3):
    cats = ' '.join([c for c in [cat2, cat3] if isinstance(c, str)]).lower()
    return any(tok in cats for tok in SKIP_CATS)

# ---------- 4차 보완 파서 ----------
def parse_supplement_facts_v4(text: str, cat2=None, cat3=None) -> dict:
    # 비-보충제 카테고리면 skip 처리 (실패로 세지 않음)
    if is_skip_category(cat2, cat3):
        return {"status": "skip", "reason": "non-supplement category", "data": None}

    if not text or not isinstance(text, str) or text.strip() == "":
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}

    block = normalize_panel(text)

    # "Supplement Facts" 키워드가 없더라도 단위가 하나도 없으면 실패
    if ("supplement facts" not in block.lower()) and (re.search(rf"\b{UNITS_PATTERN}\b", block, re.I) is None):
        return {"status": "fail", "reason": "핵심 키워드/단위 없음", "data": None}

    items = parse_line_items(block)
    if not items:
        return {"status": "fail", "reason": "성분 추출 실패", "data": None}

    return {"status": "success", "count": len(items), "data": items}

# ---------- 통합 적용 함수 ----------
def apply_final_parsing_v2(df: pd.DataFrame) -> pd.DataFrame:
    parsed_results, parse_error_flags, fail_reasons = [], [], []

    for _, row in df.iterrows():
        res = parse_supplement_facts_v4(
            row.get('Supplement Facts', ''),
            row.get('Category 2'), row.get('Category 3')
        )
        parsed_results.append(res.get('data'))
        # success/skip 은 오류 아님
        parse_error_flags.append(res['status'] == 'fail')
        fail_reasons.append(res.get('reason'))

    out = df.copy()
    out['final_parsed'] = parsed_results
    out['final_parse_error'] = parse_error_flags
    out['fail_reason_final'] = fail_reasons
    return out

# ---------- 실행 예시 (당신의 변수명에 맞춰 그대로 사용) ----------
# 1) 초기 실패만 추출 (이미 가지고 있는 df_supp_checked 기준)
df_failed = df_supp_checked[df_supp_checked['parse_error']].copy()

# 2) 4차 보완 파싱 적용
df_final_v4 = apply_final_parsing_v2(df_failed)

# 3) 최종 실패 건수/사유 확인
df_final_v4_errors = df_final_v4[df_final_v4['final_parse_error']]
print(f"❌ 최종 실패 수: {len(df_final_v4_errors)}")
print("\n📊 실패 사유 분포:")
print(df_final_v4_errors['fail_reason_final'].value_counts(dropna=False))

# 4) 실패 데이터만 CSV로 저장 (현재 작업 폴더에 저장)
df_final_v4_errors[['Title','Category 2','Category 3','Supplement Facts','fail_reason_final']].to_csv(
    'supplement_parse_errors_final_v4.csv', index=False
)
print("\n💾 저장 완료: supplement_parse_errors_final_v4.csv")

-복잡한 블랜드나 표기 단위생략,불규칙한 경우는 여전히 파싱이 안된다.

In [ ]:
# 4차 보완 파싱 결과 총 실패 20개로 줄어듦 
def parse_supplement_facts_v4(text: str) -> dict:
    import re

    if not text or not isinstance(text, str) or text.strip() == "":
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}

    block = re.sub(r'[\n\r\t]', ' ', text)
    block = re.sub(r'†|‡|[*]+|[%]+', '', block)
    block = re.sub(r'\s{2,}', ' ', block).strip()

    unit_pattern = r'(mcg|mg|g|iu|IU|CFU|DPPU|FIP|HUT|GalU|AGU|SU|CU|DP|XU|ALU|μg|ml|%)?'
    pattern = rf'([A-Za-z0-9®™\(\)\[\],\-&\/\'° +]+?)\s+([\d\.,]+)\s*{unit_pattern}'

    matches = re.findall(pattern, block)
    if not matches:
        return {"status": "fail", "reason": "성분 추출 실패", "data": None}

    nutrients = []
    for name, amount, unit in matches:
        try:
            nutrients.append({
                "name": name.strip(),
                "amount": float(amount.replace(",", "")),
                "unit": unit or ""
            })
        except:
            continue

    return {
        "status": "success" if nutrients else "fail",
        "reason": None if nutrients else "성분 추출 실패",
        "count": len(nutrients),
        "data": nutrients if nutrients else None
    }

In [ ]:
# 4차 통합 적용 함수 
def apply_final_parsing_v4(df):
    parsed_results = []
    parse_status = []
    fail_reasons = []

    for text in df['Supplement Facts']:
        result = parse_supplement_facts_v4(text)
        parsed_results.append(result['data'])
        parse_status.append(result['status'] == 'fail')
        fail_reasons.append(result['reason'] if result['status'] == 'fail' else None)

    df_result = df.copy()
    df_result['final_parsed'] = parsed_results
    df_result['final_parse_error'] = parse_status
    df_result['fail_reason_final'] = fail_reasons

    return df_result

In [ ]:
# 4) 실패 데이터만 CSV로 저장 (현재 작업 폴더에 저장)
df_final_v4_errors[['Title','Category 2','Category 3','Supplement Facts','fail_reason_final']].to_csv(
    'supplement_parse_errors_final_v4.csv', index=False
)

In [ ]:


# ✅ 5차 보완 파싱 함수
def parse_supplement_facts_v5(text: str) -> dict:
    if not text or not isinstance(text, str) or text.strip() == "":
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}

    # 1️⃣ 문자열 전처리
    cleaned = text.strip()
    block = re.sub(r'[\n\r\t]', ' ', cleaned)
    block = re.sub(r'†|‡|[*]+|[%]+|♦|•|●|◆|…|–|—', '', block)
    block = re.sub(r'\s{2,}', ' ', block).strip()

    # 2️⃣ 정규식 - 성분명에 괄호/상표 포함, 단위 다양화
    pattern = r'([A-Za-z0-9 \-–®™\(\)\[\],\'+°]+?)\s+([\d\.,]+)\s*(mg|mcg|g|IU|iu|billion CFU|CFU|ALU|HUT|XU|DP|SU|CU|FIP|DPPU|%)?'

    matches = re.findall(pattern, block)
    if not matches:
        return {"status": "fail", "reason": "성분 추출 실패", "data": None}

    nutrients = []
    for name, amount, unit in matches:
        try:
            nutrients.append({
                "name": name.strip(),
                "amount": float(amount.replace(",", "")),
                "unit": unit or ""
            })
        except:
            continue

    return {
        "status": "success" if nutrients else "fail",
        "reason": None if nutrients else "성분 추출 실패",
        "count": len(nutrients),
        "data": nutrients if nutrients else None
    }

# ✅ 통합 적용 함수
def apply_final_parsing_v5(df):
    parsed_results = []
    parse_status = []
    fail_reasons = []

    for text in df['Supplement Facts']:
        result = parse_supplement_facts_v5(text)
        parsed_results.append(result['data'])
        parse_status.append(result['status'] == 'fail')
        fail_reasons.append(result['reason'] if result['status'] == 'fail' else None)

    df_result = df.copy()
    df_result['final5_parsed'] = parsed_results
    df_result['final5_parse_error'] = parse_status
    df_result['fail_reason_final5'] = fail_reasons

    return df_result

In [ ]:
# CSV 로드
df_failed_v4 = pd.read_csv("/Users/gim-yujin/Desktop/pjt_personal_agent/supplement_parse_errors_final_v4.csv")

# 5차 파싱 적용
df_final5 = apply_final_parsing_v5(df_failed_v4)

# 최종 실패 데이터 추출
df_final5_errors = df_final5[df_final5['final5_parse_error']]

# 개수 확인
print(f"❌ 5차 파싱 후 최종 실패 수: {len(df_final5_errors)}개")

# 저장
df_final5_errors.to_csv("supplement_parse_errors_final5.csv", index=False)
print("✅ 최종 실패 데이터 저장 완료: supplement_parse_errors_final5.csv")

In [ ]:
#6차 파싱 함수 
def parse_supplement_facts_v6(text: str) -> dict:
    if not text or not isinstance(text, str) or text.strip() == "":
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}

    cleaned = text.strip()
    block = re.sub(r'[\n\r\t]', ' ', cleaned)
    block = re.sub(r'[†‡*%∞®™→•♦–•]', '', block)
    block = re.sub(r'\s{2,}', ' ', block).strip()

    # 괄호 안 숫자+단위 제거 (예: (400 mg))
    block = re.sub(r'\([^\)]*\d+(?:\.\d+)?\s?(mg|mcg|g|IU|iu|%)\)', '', block)

    # Proprietary Blend 제거 블럭 (있으면 따로 처리 가능하지만 일단 제거)
    block = re.sub(r'Proprietary Blend.*?(?=\d+[\s]*(mg|mcg|g|IU|iu|%|$))', '', block, flags=re.IGNORECASE)

    # 최종 정규식 (단위 확장 포함)
    pattern = r'([A-Za-z0-9 \-–®™\(\)\[\],\'°+/&·:]+?)\s+([\d\.,]+)\s*(billion CFU|CFU|DPPU|DU|XU|DP|ALU|HUT|SU|CU|FIP|mg|mcg|g|IU|iu|%)?'

    matches = re.findall(pattern, block)
    if not matches:
        return {"status": "fail", "reason": "성분 추출 실패", "data": None}

    nutrients = []
    for name, amount, unit in matches:
        try:
            nutrients.append({
                "name": name.strip(),
                "amount": float(amount.replace(",", "")),
                "unit": unit or ""
            })
        except:
            continue

    return {
        "status": "success" if nutrients else "fail",
        "reason": None if nutrients else "성분 추출 실패",
        "count": len(nutrients),
        "data": nutrients if nutrients else None
    }

In [ ]:
# 실패 데이터에서만 적용
df_failed5 = df_final5[df_final5['final5_parse_error']]

# 적용
parsed6 = df_failed5['Supplement Facts'].apply(parse_supplement_facts_v6)
df_failed5['final6_parsed'] = parsed6.apply(lambda x: x['data'])
df_failed5['final6_parse_error'] = parsed6.apply(lambda x: x['status'] == 'fail')
df_failed5['fail_reason_final6'] = parsed6.apply(lambda x: x['reason'] if x['status'] == 'fail' else None)

# 다시 병합
df_final6 = df_final5.copy()
df_final6.loc[df_failed5.index, 'final6_parsed'] = df_failed5['final6_parsed']
df_final6.loc[df_failed5.index, 'final6_parse_error'] = df_failed5['final6_parse_error']
df_final6.loc[df_failed5.index, 'fail_reason_final6'] = df_failed5['fail_reason_final6']

In [ ]:
# NaN 값을 False로 간주하도록 처리
df_fail_final6 = df_final6[df_final6['final6_parse_error'].fillna(False)]

# 결과 확인
print(f"🚨 최종 6차 파싱 실패 수: {len(df_fail_final6)}개")

# CSV 저장
df_fail_final6.to_csv('supplement_parse_errors_final6.csv', index=False)

In [ ]:
## 7차 파싱 함수
import re

def parse_supplement_facts_v7(text: str) -> dict:
    if not text or not isinstance(text, str) or text.strip() == "":
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}

    # 1. 텍스트 정제
    block = re.sub(r'[\n\r\t]', ' ', text.strip())
    block = re.sub(r'†|‡|[*]+|[%]+', '', block)
    block = re.sub(r'\s{2,}', ' ', block).strip()

    # 2. 문자+숫자 붙어있는 경우 공백 넣기 (예: "VitaminC1000mg" → "VitaminC 1000mg")
    block = re.sub(r'([a-zA-Z\)])(?=\d)', r'\1 ', block)

    # 3. 패턴 정의
    pattern = r'([A-Za-z0-9 \-–®™\(\)\[\],\'+°]+?)\s+([\d\.,]+)\s*(mcg|mg|g|iu|IU|%)?'

    matches = re.findall(pattern, block)
    if not matches:
        return {"status": "fail", "reason": "성분 추출 실패", "data": None}

    nutrients = []
    for name, amount, unit in matches:
        try:
            nutrients.append({
                "name": name.strip(),
                "amount": float(amount.replace(",", "")),
                "unit": unit or ""
            })
        except:
            continue

    return {
        "status": "success" if nutrients else "fail",
        "reason": None if nutrients else "성분 추출 실패",
        "count": len(nutrients),
        "data": nutrients if nutrients else None
    }

In [ ]:
# 통합 적용 함수 
def apply_final7_parsing(df):
    parsed_results = []
    parse_status = []
    fail_reasons = []

    for text in df['Supplement Facts']:
        result = parse_supplement_facts_v7(text)
        parsed_results.append(result['data'])
        parse_status.append(result['status'] == 'fail')
        fail_reasons.append(result['reason'] if result['status'] == 'fail' else None)

    df_result = df.copy()
    df_result['final7_parsed'] = parsed_results
    df_result['final7_parse_error'] = parse_status
    df_result['fail_reason_final7'] = fail_reasons

    return df_result

In [ ]:
# 1️⃣ 이전까지 실패한 데이터 불러오기
df_fail_final6 = df_final6[df_final6['final6_parse_error'].fillna(False)]

# 2️⃣ 7차 파싱 적용
df_final7 = apply_final7_parsing(df_fail_final6)

# 3️⃣ 파싱 실패한 데이터만 추출
df_fail_final7 = df_final7[df_final7['final7_parse_error'].fillna(False)]

# 4️⃣ 개수 출력
print(f"🚨 최종 7차 파싱 실패 수: {len(df_fail_final7)}개")

# 5️⃣ CSV 저장
df_fail_final7.to_csv('supplement_parse_errors_final7.csv', index=False)

# 전체 통합 정리 코드 

In [ ]:
# 1️⃣ 전체 데이터프레임: df_supp_checked 는 초기 전체 325개 데이터라고 가정
df_failed = df_supp_checked[df_supp_checked['parse_error']]  # 1차 파싱 실패

In [ ]:
# 예시: 7차 파싱 함수 (최신 버전으로 대체)
def parse_supplement_facts_v7(text):
    import re
    if not text or not isinstance(text, str) or text.strip() == "":
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}
    
    block = re.sub(r'[\n\r\t]', ' ', text.strip())
    block = re.sub(r'[†‡*%]+', '', block)
    block = re.sub(r'\s{2,}', ' ', block)

    pattern = r'([A-Za-z0-9 \-–®™\(\)\[\],\'+°]+?)\s+([\d\.,]+)\s*(mcg|mg|g|iu|IU|%)?'
    matches = re.findall(pattern, block)

    nutrients = []
    for name, amount, unit in matches:
        try:
            nutrients.append({
                "name": name.strip(),
                "amount": float(amount.replace(",", "")),
                "unit": unit or ""
            })
        except:
            continue

    return {
        "status": "success" if nutrients else "fail",
        "reason": None if nutrients else "성분 추출 실패",
        "data": nutrients if nutrients else None
    }

# ✅ 통합 적용 함수
def apply_final_parsing_v7(df):
    parsed_results = []
    parse_status = []
    fail_reasons = []

    for text in df['Supplement Facts']:
        result = parse_supplement_facts_v7(text)
        parsed_results.append(result['data'])
        parse_status.append(result['status'] == 'fail')
        fail_reasons.append(result['reason'] if result['status'] == 'fail' else None)

    df_result = df.copy()
    df_result['final7_parsed'] = parsed_results
    df_result['final7_parse_error'] = parse_status
    df_result['fail_reason_final7'] = fail_reasons

    return df_result

# ✅ 7차 파싱 실행
df_final7 = apply_final_parsing_v7(df_failed)

In [ ]:
# 전체 DataFrame 복사
df_total = df_supp_checked.copy()

# 컬럼 초기화 (초기값은 기존 결과)
df_total['final_parsed']      = df_total.get('parsed_ingredients', None)
df_total['final_parse_error'] = df_total.get('parse_error', None)
df_total['final_fail_reason'] = df_total.get('fail_reason', None)

# 7차 파싱된 인덱스를 기준으로 덮어쓰기
idx = df_final7.index
df_total.loc[idx, 'final_parsed']      = df_final7['final7_parsed']
df_total.loc[idx, 'final_parse_error'] = df_final7['final7_parse_error']
df_total.loc[idx, 'final_fail_reason'] = df_final7['fail_reason_final7']

# 성공한 경우 실패 사유 제거
df_total.loc[df_total['final_parse_error'] == False, 'final_fail_reason'] = None

In [281]:
# 파싱 성공 / 실패 / 누락 개수 확인
total = len(df_total)
num_success = (df_total['final_parse_error'] == False).sum()
num_fail = (df_total['final_parse_error'] == True).sum()
num_na = df_total['final_parse_error'].isna().sum()

print(f"📊 전체 레코드 수: {total}")
print(f"✅ 파싱 성공: {num_success}개")
print(f"❌ 파싱 실패: {num_fail}개")
print(f"❓ 상태 미확인 (NaN): {num_na}개")


📊 전체 레코드 수: 325
✅ 파싱 성공: 214개
❌ 파싱 실패: 111개
❓ 상태 미확인 (NaN): 0개


In [ ]:
# 파싱 완료된 전체 데이터 저장
df_total.to_csv("supplement_parsing_total_final.csv", index=False)

# 파싱 실패 데이터만 저장
df_total[df_total['final_parse_error'] == True].to_csv("supplement_parsing_failed_final.csv", index=False)

In [ ]:
# 인덱스 정렬 후 병합 (권장)
df_total['fail_reason'] = df_final7['fail_reason_final7'].reindex(df_total.index)

In [ ]:
if 'fail_reason' not in df_total.columns:
    print("⚠️ 'fail_reason' 컬럼이 없습니다. 먼저 추가해 주세요.")
else:
    print(df_total['fail_reason'].value_counts())

In [ ]:
# 성분 추출 실패한 92개 개선 
def parse_supplement_facts_v8(text: str) -> dict:
    import re
    if not text or not isinstance(text, str) or text.strip() == "":
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}

    block = re.sub(r'[\n\r\t]', ' ', text.strip())
    block = re.sub(r'[†‡*%]+', '', block)
    block = re.sub(r'\s{2,}', ' ', block)

    # 좀 더 자유롭게 괄호와 단위 허용
    pattern = r'([A-Za-z0-9 \-–®™\(\)\[\],\'°©®]+?)\s*[:\-]?\s*([\d\.,]+)\s*(mcg|mg|g|iu|IU|ml|%)?'

    matches = re.findall(pattern, block)

    if not matches:
        return {"status": "fail", "reason": "성분 추출 실패", "data": None}

    nutrients = []
    for name, amount, unit in matches:
        try:
            nutrients.append({
                "name": name.strip(),
                "amount": float(amount.replace(',', '')),
                "unit": unit or ''
            })
        except:
            continue

    return {
        "status": "success" if nutrients else "fail",
        "reason": None if nutrients else "성분 추출 실패",
        "data": nutrients,
        "count": len(nutrients)
    }

In [ ]:
df_fail = df_total[df_total['final_parse_error']]
df_retry8 = df_fail.copy()

# 새로 파싱 시도
results = df_retry8['Supplement Facts'].apply(parse_supplement_facts_v8)

df_retry8['final8_parsed'] = results.map(lambda x: x['data'])
df_retry8['final8_parse_error'] = results.map(lambda x: x['status'] == 'fail')
df_retry8['fail_reason_final8'] = results.map(lambda x: x['reason'])

In [ ]:
# 먼저 index가 일치하는지 확인 (안하면 오류납니다!)
df_retry8 = df_retry8.copy()
df_retry8 = df_retry8[['final8_parsed', 'final8_parse_error', 'fail_reason_final8']]

# 원본 df_total에 있는 실패 데이터의 인덱스 기준으로 업데이트
df_total.loc[df_retry8.index, 'parsed_data'] = df_retry8['final8_parsed']
df_total.loc[df_retry8.index, 'parse_error'] = df_retry8['final8_parse_error']
df_total.loc[df_retry8.index, 'fail_reason'] = df_retry8['fail_reason_final8']

In [ ]:
# 파싱 성공/실패/NaN 통계 요약
total = len(df_total)
num_success = (df_total['parse_error'] == False).sum()
num_fail = (df_total['parse_error'] == True).sum()
num_na = df_total['parse_error'].isna().sum()

print(f"📊 전체 레코드 수: {total}")
print(f"✅ 파싱 성공: {num_success}개")
print(f"❌ 파싱 실패: {num_fail}개")
print(f"❓ 상태 미확인 (NaN): {num_na}개") 

📊 전체 레코드 수: 325
✅ 파싱 성공: 219개
❌ 파싱 실패: 106개
❓ 상태 미확인 (NaN): 0개


In [ ]:
fail_summary = df_total[df_total['parse_error'] == True]['fail_reason'].value_counts()
print("\n📉 파싱 실패 사유 분포:")
print(fail_summary)

In [ ]:
# 실패한 레코드 중에서 실패 사유가 비어 있는 (NaN) 데이터
df_fail = df_total[df_total['parse_error'] == True]
df_fail_nan_reason = df_fail[df_fail['fail_reason'].isna()]

print(f"❓ 실패했지만 실패 사유가 없는 레코드 수: {len(df_fail_nan_reason)}개")

In [ ]:
# 실패 사유가 없는 데이터만 저장
df_fail_nan_reason.to_csv('supplement_parsing_failed_reason_missing.csv', index=False)
print("📁 supplement_parsing_failed_reason_missing.csv 파일로 저장 완료")

In [269]:
# 파싱 실패한 106개 레코드만 필터링
df_fail = df_total[df_total['parse_error'] == True]

# 컬럼 목록 출력
print("📋 파싱 실패한 데이터프레임의 컬럼 목록:")
print(df_fail.columns.tolist())

📋 파싱 실패한 데이터프레임의 컬럼 목록:
['P url', 'Title', 'Product code', 'Price', 'Currency', 'UPC Code', 'Pid', 'Shipping Weight', 'Dimensions', 'Category 1', 'Category 2', 'Category 3', 'Availability', 'Brand', 'images', 'Product specifications', 'Description', 'ingredients', 'Supplement Facts', 'Avg rating', 'Reviews count', 'uniq_id', 'scraped_at', 'parsed_ingredients', 'parse_error', 'final_parsed', 'final_parse_error', 'final_fail_reason', 'fail_reason', 'parsed_data']


In [282]:
print(df_total.columns)

Index(['P url', 'Title', 'Product code', 'Price', 'Currency', 'UPC Code',
       'Pid', 'Shipping Weight', 'Dimensions', 'Category 1', 'Category 2',
       'Category 3', 'Availability', 'Brand', 'images',
       'Product specifications', 'Description', 'ingredients',
       'Supplement Facts', 'Avg rating', 'Reviews count', 'uniq_id',
       'scraped_at', 'parsed_ingredients', 'parse_error', 'final_parsed',
       'final_parse_error', 'final_fail_reason', 'fail_reason', 'parsed_data'],
      dtype='object')


In [301]:
df_parsed = df_total[df_total['final_parse_error'] == False]

In [ ]:
# 즉, **parse_error 컬럼을 기준으로 했기 때문에 219개**로 나왔던 것이고,
# final_parse_error 기준은 214개입니다

len(df_parsed)

214

In [302]:
# ✅ 최종 파싱 성공한 데이터만 필터링
df_parsed = df_total[df_total['final_parse_error'] == False]

# ✅ CSV 파일로 저장
df_parsed.to_csv("parsed_supplements_final.csv", index=False)
print("✅ 파싱된 데이터가 'parsed_supplements_final.csv'로 저장되었습니다.")

✅ 파싱된 데이터가 'parsed_supplements_final.csv'로 저장되었습니다.


In [304]:
df_parsed

,P url,Title,Product code,Price,Currency,UPC Code,Pid,Shipping Weight,Dimensions,Category 1,...,Reviews count,uniq_id,scraped_at,parsed_ingredients,parse_error,final_parsed,final_parse_error,final_fail_reason,fail_reason,parsed_data
6,https://uk.iherb.com/pr/phillip-s-laxative-cap...,"Phillip's, Laxative Caplets, 55 Caplets",312843520820,13.05,GBP,PHP-35208,108793,0.12 kg,10.5 x 7.9 x 5.5 cm,Phillips,...,70,997186b1-a3c2-5f99-a0e6-af22a78d2651,2022-12-15 12:39:41,"[{'name': 'Daily ValueMagnesium', 'amount': 50...",False,"[{'name': 'Daily ValueMagnesium', 'amount': 50...",False,None,NaN,NaN
11,https://uk.iherb.com/pr/carlson-kid-s-vitamin-...,"Carlson, Kid's Vitamin C Gummies, Natural Oran...",88395490309,12.77,GBP,CAR-49030,86463,0.25 kg,6.4 x 6.4 x 11.7 cm,Carlson,...,337,e48dfd2b-f328-5be5-bc6a-a28a29a07dd1,2022-12-15 12:39:51,"[{'name': 'Includes', 'amount': 2.0, 'unit': '...",False,"[{'name': 'Includes', 'amount': 2.0, 'unit': '...",False,None,NaN,NaN
24,https://uk.iherb.com/pr/now-foods-glycine-pure...,"NOW Foods, Glycine, Pure Powder, 1 lb (454 g)",733739002259,21.64,GBP,NOW-00225,615,0.62 kg,16.5 x 8.6 x 8.6 cm,NOW Foods,...,2640,e760a661-6ac1-573d-8377-ea5c8f243dc8,2022-12-15 12:40:13,[],True,"[{'name': 'about', 'amount': 151.0, 'unit': ''...",False,None,None,NaN
25,https://uk.iherb.com/pr/sun-chlorella-chlorell...,"Sun Chlorella, Chlorella Supplement, 200 mg, 3...",29918011113,34.31,GBP,SCC-01111,34165,0.11 kg,2.5 x 11.9 x 10.2 cm,Sun Chlorella,...,106,f8c4249d-8234-572d-b7b0-f5b81e985146,2022-12-15 12:40:13,"[{'name': 'Total CarbohydrateLess than', 'amou...",False,"[{'name': 'Total CarbohydrateLess than', 'amou...",False,None,NaN,NaN
29,https://uk.iherb.com/pr/flora-certified-organi...,"Flora, Certified Organic Elderberry + With Ech...",61998628041,15.64,GBP,FLO-62804,101959,0.72 kg,13.7 x 6.1 x 6.1 cm,Flora,...,288,168f97b9-2bfd-5ce6-899d-55721b82e2d6,2022-12-15 12:40:18,[{'name': 'Daily ValueOrganic Black Elderberry...,False,[{'name': 'Daily ValueOrganic Black Elderberry...,False,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,https://uk.iherb.com/pr/celestial-seasonings-h...,"Celestial Seasonings, Herbal Tea, Caffeine Fre...",70734053351,4.24,GBP,CES-05335,10677,0.13 kg,6.4 x 13.7 x 7.6 cm,Celestial Seasonings,...,1544,ad474ac9-eb4d-53e2-b150-c1345d669b5b,2022-12-15 14:22:08,[],True,"[{'name': '1 Tea Bag (2g) Makes', 'amount': 8....",False,None,None,NaN
1233,https://uk.iherb.com/pr/new-chapter-fermented-...,"New Chapter, Fermented Vitamin B Complex, 90 V...",727783903594,45.95,GBP,NCR-90359,114128,0.29 kg,11.2 x 6.2 x 6 cm,New Chapter,...,106,574a8ef4-8a92-59af-8095-f9b96897a06b,2022-12-15 14:22:12,[{'name': 'Daily ValueThiamin (as thiamine hyd...,False,[{'name': 'Daily ValueThiamin (as thiamine hyd...,False,None,NaN,NaN
1237,https://uk.iherb.com/pr/natures-craft-uric-aci...,"Natures Craft, Uric Acid Formula, 60 Capsules",806808278722,30.05,GBP,NCA-27872,114209,0.13 kg,11 x 5.8 x 5.8 cm,Natures Craft,...,5,50551c62-9a40-5a46-94cb-3015ee3584ab,2022-12-15 14:22:25,[],True,[{'name': 'Amount Per ServingDaily ValueVitami...,False,None,None,NaN
1238,https://uk.iherb.com/pr/chapter-one-o-is-for-o...,"Chapter One, O is for Omega, Flavored Gummies,...",848998041069,18.11,GBP,CTO-04106,107159,0.27 kg,11.7 x 6.1 x 6.1 cm,Chapter One,...,58,6849c64c-f98b-58d8-a5cd-2b14921e6371,2022-12-15 14:22:29,"[{'name': 'Total Carbohydrate', 'amount': 5.0,...",False,"[{'name': 'Total Carbohydrate', 'amount': 5.0,...",False,None,NaN,NaN


In [300]:
# 예: 인덱스 6번의 전체 데이터 보기
import pprint
pprint.pprint(df_parsed['final_parsed'].iloc[0])

[{'amount': 500.0,
  'name': 'Daily ValueMagnesium',
  'raw_line': 'Supplement FactsServing Size:\xa01 Caplet\xa0Amount Per '
              'Serving%Daily ValueMagnesium\xa0500 mg119%',
  'unit': 'mg'}]


In [305]:
# ✅ parsed_data 컬럼 제거
df_parsed.drop(columns=['parsed_data'], inplace=True)

/var/folders/98/vj3q9b254k79y681ng12dddh0000gn/T/ipykernel_86978/2878155958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parsed.drop(columns=['parsed_data'], inplace=True)


In [306]:
df_parsed.to_csv("parsed_supplements_final_cleaned.csv", index=False)
print("✅ cleaned 파일 저장 완료")

✅ cleaned 파일 저장 완료


In [307]:
df_parsed

,P url,Title,Product code,Price,Currency,UPC Code,Pid,Shipping Weight,Dimensions,Category 1,...,Avg rating,Reviews count,uniq_id,scraped_at,parsed_ingredients,parse_error,final_parsed,final_parse_error,final_fail_reason,fail_reason
6,https://uk.iherb.com/pr/phillip-s-laxative-cap...,"Phillip's, Laxative Caplets, 55 Caplets",312843520820,13.05,GBP,PHP-35208,108793,0.12 kg,10.5 x 7.9 x 5.5 cm,Phillips,...,4.5,70,997186b1-a3c2-5f99-a0e6-af22a78d2651,2022-12-15 12:39:41,"[{'name': 'Daily ValueMagnesium', 'amount': 50...",False,"[{'name': 'Daily ValueMagnesium', 'amount': 50...",False,None,NaN
11,https://uk.iherb.com/pr/carlson-kid-s-vitamin-...,"Carlson, Kid's Vitamin C Gummies, Natural Oran...",88395490309,12.77,GBP,CAR-49030,86463,0.25 kg,6.4 x 6.4 x 11.7 cm,Carlson,...,4.9,337,e48dfd2b-f328-5be5-bc6a-a28a29a07dd1,2022-12-15 12:39:51,"[{'name': 'Includes', 'amount': 2.0, 'unit': '...",False,"[{'name': 'Includes', 'amount': 2.0, 'unit': '...",False,None,NaN
24,https://uk.iherb.com/pr/now-foods-glycine-pure...,"NOW Foods, Glycine, Pure Powder, 1 lb (454 g)",733739002259,21.64,GBP,NOW-00225,615,0.62 kg,16.5 x 8.6 x 8.6 cm,NOW Foods,...,4.7,2640,e760a661-6ac1-573d-8377-ea5c8f243dc8,2022-12-15 12:40:13,[],True,"[{'name': 'about', 'amount': 151.0, 'unit': ''...",False,None,None
25,https://uk.iherb.com/pr/sun-chlorella-chlorell...,"Sun Chlorella, Chlorella Supplement, 200 mg, 3...",29918011113,34.31,GBP,SCC-01111,34165,0.11 kg,2.5 x 11.9 x 10.2 cm,Sun Chlorella,...,4.8,106,f8c4249d-8234-572d-b7b0-f5b81e985146,2022-12-15 12:40:13,"[{'name': 'Total CarbohydrateLess than', 'amou...",False,"[{'name': 'Total CarbohydrateLess than', 'amou...",False,None,NaN
29,https://uk.iherb.com/pr/flora-certified-organi...,"Flora, Certified Organic Elderberry + With Ech...",61998628041,15.64,GBP,FLO-62804,101959,0.72 kg,13.7 x 6.1 x 6.1 cm,Flora,...,4.7,288,168f97b9-2bfd-5ce6-899d-55721b82e2d6,2022-12-15 12:40:18,[{'name': 'Daily ValueOrganic Black Elderberry...,False,[{'name': 'Daily ValueOrganic Black Elderberry...,False,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,https://uk.iherb.com/pr/celestial-seasonings-h...,"Celestial Seasonings, Herbal Tea, Caffeine Fre...",70734053351,4.24,GBP,CES-05335,10677,0.13 kg,6.4 x 13.7 x 7.6 cm,Celestial Seasonings,...,4.6,1544,ad474ac9-eb4d-53e2-b150-c1345d669b5b,2022-12-15 14:22:08,[],True,"[{'name': '1 Tea Bag (2g) Makes', 'amount': 8....",False,None,None
1233,https://uk.iherb.com/pr/new-chapter-fermented-...,"New Chapter, Fermented Vitamin B Complex, 90 V...",727783903594,45.95,GBP,NCR-90359,114128,0.29 kg,11.2 x 6.2 x 6 cm,New Chapter,...,4.7,106,574a8ef4-8a92-59af-8095-f9b96897a06b,2022-12-15 14:22:12,[{'name': 'Daily ValueThiamin (as thiamine hyd...,False,[{'name': 'Daily ValueThiamin (as thiamine hyd...,False,None,NaN
1237,https://uk.iherb.com/pr/natures-craft-uric-aci...,"Natures Craft, Uric Acid Formula, 60 Capsules",806808278722,30.05,GBP,NCA-27872,114209,0.13 kg,11 x 5.8 x 5.8 cm,Natures Craft,...,5.0,5,50551c62-9a40-5a46-94cb-3015ee3584ab,2022-12-15 14:22:25,[],True,[{'name': 'Amount Per ServingDaily ValueVitami...,False,None,None
1238,https://uk.iherb.com/pr/chapter-one-o-is-for-o...,"Chapter One, O is for Omega, Flavored Gummies,...",848998041069,18.11,GBP,CTO-04106,107159,0.27 kg,11.7 x 6.1 x 6.1 cm,Chapter One,...,4.7,58,6849c64c-f98b-58d8-a5cd-2b14921e6371,2022-12-15 14:22:29,"[{'name': 'Total Carbohydrate', 'amount': 5.0,...",False,"[{'name': 'Total Carbohydrate', 'amount': 5.0,...",False,None,NaN


## 컬럼명 정리
final_parsed
✅ 최종 파싱 결과 리스트 ([{name, amount, unit, ...}, ...] 형태). 실제로 Supplement Facts를 정규표현식으로 파싱한 결과입니다.
final_parse_error
✅ 파싱 성공 여부를 담은 True/False 값. True면 파싱 실패, False면 성공입니다.
final_fail_reason
✅ 파싱 실패 이유를 설명하는 텍스트. 예: "성분 추출 실패", "공란 또는 타입 오류" 등
fail_reason
(중간 버전에서 사용하던) 파싱 실패 사유. 지금은 final_fail_reason이 최신이므로 이건 거의 NaN 상태일 수 있어요.
parsed_data
❌ 초기 파싱 때 쓰던 결과 컬럼인데, 지금은 쓰지 않아요. 최종 결과는 final_parsed로 대체되었습니다.
